## Global Black Box Optimization with RNNS

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def kernel(x1,x2,l = 0.3):
    return np.exp(-1.0/l**2*np.sum((np.expand_dims(x1,axis=2) - np.expand_dims(x2,axis=1))**2, axis = 3))

def GP(X,A,x, l = 0.3):
    k_xX = kernel(x,X)
    return np.squeeze(np.matmul(k_xX,  A),axis=(2,))

def kernelTF(x1,x2,l = 0.3):
    return tf.exp(-1.0/l**2*tf.reduce_sum((tf.expand_dims(x1,axis=2) - tf.expand_dims(x2,axis=1))**2, axis = 3))

def GPTF(X,A,x, l = 0.3):
    k_xX = kernelTF(tf.expand_dims(x, axis = 1),X)
    return tf.tanh(tf.squeeze(tf.matmul(k_xX,  A),axis=(2,)))

In [3]:
def gen_data(n_train, n_test, dim, n_gp_samples):
    import itertools as it
    X = np.tile(np.array(list(it.product(np.linspace(-1,1,n_gp_samples),repeat=dim))), (n_train+n_test,1,1))
    Y = np.random.uniform(low = -1.0, high = 1.0, size = (n_train+n_test, n_gp_samples**dim))

    K_XX = kernel(X,X)
    A = np.linalg.solve(K_XX, np.expand_dims(Y,axis=2))

    return (X[:n_train], Y[:n_train], A[:n_train], X[-n_test:], Y[-n_test:], A[-n_test:])

In [4]:
# Data
n_train = 1600
n_test = 16
n_gp_samples = 3
dim = 1

# LSTM Model
n_hidden = 50
n_steps = 10

# Optimization
learning_rate = 0.0001
epochs = 200
batch_size = 160

In [5]:
X_train, Y_train, A_train, X_test, Y_test, A_test = gen_data(n_train, n_test, dim, n_gp_samples)

In [6]:
# LSTM Output Weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, dim]))
}
biases = {
    'out': tf.Variable(tf.random_normal([dim]))
}

In [7]:
# Create Model
size = tf.placeholder(tf.int32,[])

Xt = tf.placeholder(tf.float32, [None, n_gp_samples**dim, dim])
At = tf.placeholder(tf.float32, [None, n_gp_samples**dim, 1])

x_0 = -0.0*tf.ones([size, dim])
h_0 = tf.ones([size, n_hidden])
c_0 = tf.ones([size, n_hidden])

state = (c_0, h_0)
x = x_0
y = GPTF(Xt,At,x)
sample_points = [x]

f_min = y
f_sum = 0

# No idea why this is necessary 
cell = tf.contrib.rnn.LSTMCell(num_units = n_hidden, reuse=None)
cell(tf.concat([x, y], 1), state, scope='rnn_cell')
cell = tf.contrib.rnn.LSTMCell(num_units = n_hidden, reuse=True)

for i in range(n_steps):
    h, state = cell(tf.concat([x, y], 1), state, scope='rnn_cell')
    x = tf.tanh(tf.matmul(h, weights['out']) + biases['out'])
    sample_points.append(x)
    
    y = GPTF(Xt,At,x)
    
    f_min = tf.minimum(y, f_min)
    f_sum += tf.reduce_mean(y)

f_min = tf.reduce_mean(f_min)
loss = f_sum / n_steps

In [8]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

train_loss_list = []
test_loss_list = []
train_fmin_list = []
test_fmin_list = []

In [10]:
# Train the Network
print("Function Dimension: \t\t"+str(dim))
print("Number of Training Samples: \t"+str(n_train))
print("Batch size: \t\t\t"+str(batch_size))
print("Number of hidden Units: \t"+str(n_hidden))
print("Sequence length: \t\t"+ str(n_steps))
print("Epochs: \t\t\t"+str(epochs))
print("Learning rate: \t\t\t"+str(learning_rate))
print("------------------------------------------------------------------------------------")

for ep in range(epochs):
    for batch in range(n_train//batch_size):
        X_batch = X_train[batch*batch_size:(batch+1)*batch_size]
        A_batch = A_train[batch*batch_size:(batch+1)*batch_size]
        
        sess.run([train_step], feed_dict={Xt: X_batch, At: A_batch, size: batch_size})
    
    train_loss, train_fmin = sess.run([loss, f_min], feed_dict={Xt: X_train, At: A_train, size: n_train})
    test_loss, test_fmin = sess.run([loss, f_min], feed_dict={Xt: X_test, At: A_test, size:n_test})
    
    train_loss_list += [train_loss]
    test_loss_list += [test_loss]
    train_fmin_list += [train_fmin]
    test_fmin_list += [test_fmin]
    
    if ep < 10 or ep % (epochs // 10) == 0 or ep == epochs-1:
        print("Ep: " +"{:4}".format(ep)+" | TrainLoss: "+"{: .3f}".format(train_loss)
              +" | TrainMin: "+ "{: .3f}".format(train_fmin)+ " | TestLoss: "+
              "{: .3f}".format(test_loss)+" | TestMin: "+ "{: .3f}".format(test_fmin))

Function Dimension: 		1
Number of Training Samples: 	1600
Batch size: 			160
Number of hidden Units: 	50
Sequence length: 		10
Epochs: 			200
Learning rate: 			0.0001
------------------------------------------------------------------------------------
Ep:    0 | TrainLoss: -0.039 | TrainMin: -0.362 | TestLoss: -0.026 | TestMin: -0.348
Ep:    1 | TrainLoss: -0.068 | TrainMin: -0.349 | TestLoss: -0.052 | TestMin: -0.335
Ep:    2 | TrainLoss: -0.073 | TrainMin: -0.349 | TestLoss: -0.059 | TestMin: -0.335
Ep:    3 | TrainLoss: -0.084 | TrainMin: -0.353 | TestLoss: -0.058 | TestMin: -0.340
Ep:    4 | TrainLoss: -0.094 | TrainMin: -0.353 | TestLoss: -0.065 | TestMin: -0.339
Ep:    5 | TrainLoss: -0.102 | TrainMin: -0.350 | TestLoss: -0.074 | TestMin: -0.337
Ep:    6 | TrainLoss: -0.112 | TrainMin: -0.351 | TestLoss: -0.075 | TestMin: -0.338
Ep:    7 | TrainLoss: -0.115 | TrainMin: -0.348 | TestLoss: -0.087 | TestMin: -0.336
Ep:    8 | TrainLoss: -0.118 | TrainMin: -0.348 | TestLoss: -0.087 |